In [1]:
import torch
import numpy as np
import nibabel as nib

In [23]:
from einops import rearrange

In [24]:
from torch.nn.functional import grid_sample

In [16]:
fix_kpts_csv = '/gpfs/space/projects/BetterMedicine/illia/bm-ai-pipelines/data/datasets/TUH/nifti_subset_reorient_false/00a49d80-12d7-eefe-c63b-0354c24aefae/fixed_kpts.csv'
mov_kpts_csv = '/gpfs/space/projects/BetterMedicine/illia/bm-ai-pipelines/data/datasets/TUH/nifti_subset_reorient_false/00a49d80-12d7-eefe-c63b-0354c24aefae/moving_kpts.csv'
disp_npy = '/gpfs/space/projects/BetterMedicine/illia/bm-ai-pipelines/data/datasets/TUH/nifti_subset_reorient_false/00a49d80-12d7-eefe-c63b-0354c24aefae/displacement.npy'


In [6]:
fix_kpts = np.loadtxt(fix_kpts_csv, delimiter=',')
mov_kpts = np.loadtxt(mov_kpts_csv, delimiter=',')

fix_kpts = fix_kpts[None, :]
mov_kpts = mov_kpts[None, :]

In [7]:
fix_kpts.shape

(1, 2303, 3)

In [8]:
mov_kpts.shape

(1, 2303, 3)

In [17]:
disp_npy = np.load(disp_npy)

In [20]:
disp_npy.shape

(1, 256, 208, 256, 3)

In [27]:
B,H,W,D = disp_npy.shape[:4]
centers = ((H-1)/2, (W-1)/2, (D-1)/2)
fix_kpts_centr = fix_kpts - centers # move the origin of coordinate system to the center of image
fix_kpts_norm = fix_kpts_centr / np.array(centers) # normalising to [-1,1]
fix_kpts_norm = fix_kpts_norm[:, :, [1, 0, 2]] # grid_sample expects the (x,y,z) values. We have (H,W,D). 
                                               # As far as I know H = y, W = x, D = z. Thus, we need to swap axes.
fix_kpts_norm = rearrange(torch.from_numpy(fix_kpts_norm.copy()), 'B N C -> B N 1 1 C').to(torch.float32) # rearrange to the expected grid_sample shape
print(fix_kpts_norm.shape)

torch.Size([1, 2303, 1, 1, 3])


In [30]:
disp_torch = rearrange(torch.from_numpy(disp_npy), 'B H W D C -> B C D H W').to(torch.float32)
disp_torch.shape

torch.Size([1, 3, 256, 256, 208])

In [31]:
torch_disp_at = grid_sample(disp_torch, fix_kpts_norm, mode='bilinear', align_corners=True, padding_mode='border')

In [32]:
torch_disp_at = rearrange(torch_disp_at, 'B C N 1 1 -> B N C')

In [33]:
disp_kpts_torch_warped = fix_kpts + torch_disp_at.numpy()

print(np.allclose(disp_kpts_torch_warped, mov_kpts, rtol=0.1, atol=0.1))

True
